# Imports

In [1]:
import csv
import mysql.connector as mysql
from concurrent.futures import ThreadPoolExecutor

# MySQL Connection

In [22]:
conn = mysql.connect(
    host="localhost",
    user="root",
    password="karenjblab2025",
    database="enem"
)

In [3]:
cursor = conn.cursor()

# Dimensional Modeling

## Dimension Tables

### dimension_enrollment

#### Data Read

In [4]:
# Leitura do arquivo CSV
data_dimension_enrollment = []
with open("C:/Users/karen/OneDrive/Desktop/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", mode="r", encoding="latin-1") as file:
    reader = csv.reader(file, delimiter=";")
    next(reader)  # Ignora o cabeçalho

    for row in reader:
        data_dimension_enrollment.append((
            row[0],  # NU_INSCRICAO
            row[1],  # NU_ANO
            row[2],  # TP_FAIXA_ETARIA
            row[3],  # TP_SEXO
            row[4],  # TP_ESTADO_CIVIL
            row[5],  # TP_COR_RACA
            row[6],  # TP_NACIONALIDADE
            row[7],  # TP_ST_CONCLUSAO
            row[8],  # TP_ANO_CONCLUIU
            row[9],  # TP_ESCOLA
            row[10], # TP_ENSINO
            row[11]  # IN_TREINEIRO
        ))

#### MySQL Create Table

In [5]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dimension_enrollment (
        NU_INSCRICAO BIGINT PRIMARY KEY,
        NU_ANO INT,
        TP_FAIXA_ETARIA INT,
        TP_SEXO CHAR(1),
        TP_ESTADO_CIVIL INT,
        TP_COR_RACA INT,
        TP_NACIONALIDADE INT,
        TP_ST_CONCLUSAO INT,
        TP_ANO_CONCLUIU INT,
        TP_ESCOLA INT,
        TP_ENSINO INT,
        IN_TREINEIRO BOOLEAN
    )
""")

In [6]:
conn.commit() # Grava todas as alterações no banco

#### MySQL Insert Data

In [7]:
chunk_size = 100000  # Inserir em lotes de 100 mil
for i in range(0, len(data_dimension_enrollment), chunk_size):
    batch = data_dimension_enrollment[i:i+chunk_size]
    cursor.executemany("""
        INSERT IGNORE INTO dimension_enrollment (
            NU_INSCRICAO, NU_ANO, TP_FAIXA_ETARIA, TP_SEXO, TP_ESTADO_CIVIL, TP_COR_RACA,
            TP_NACIONALIDADE, TP_ST_CONCLUSAO, TP_ANO_CONCLUIU, TP_ESCOLA, TP_ENSINO, IN_TREINEIRO
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, batch)
    conn.commit()


In [8]:
cursor.execute("SELECT COUNT(*) FROM dimension_enrollment;")
rows = cursor.fetchall()
print(f"Total de registros na tabela: {rows[0][0]}")

Total de registros na tabela: 5783109


### dimension_school

#### Data Read

In [9]:
def safe_int(value, default=0):
    """Converte valores para int ou retorna um valor padrão."""
    try:
        return int(value)
    except ValueError:
        return default  # Retorna o valor padrão se o valor for inválido

In [10]:
# Leitura do arquivo CSV
data_dimension_school = []
with open("C:/Users/karen/OneDrive/Desktop/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", mode="r", encoding="latin-1") as file:
    reader = csv.reader(file, delimiter=";")
    next(reader)  # Ignora o cabeçalho

    for row in reader:
        data_dimension_school.append((
            safe_int(row[0]),        # NU_INSCRICAO como BIGINT
            safe_int(row[65]),       # CO_MUNICIPIO_ESC como INT
            row[66] if row[66] else "N/A",   # NO_MUNICIPIO_ESC como VARCHAR(255), substitui vazio por "N/A"
            safe_int(row[67]),       # CO_UF_ESC como INT
            row[68] if row[68] else "N/A",   # SG_UF_ESC como CHAR(2), substitui vazio por "N/A"
            safe_int(row[69]),       # TP_DEPENDENCIA_ADM_ESC como INT
            safe_int(row[70]),       # TP_LOCALIZACAO_ESC como INT
            safe_int(row[71])        # TP_SIT_FUNC_ESC como INT
        ))

In [11]:
print(data_dimension_school[:10])  # Exibe os primeiros 10 registros

[(200006271946, 0, 'N/A', 0, 'N/A', 0, 0, 0), (200001195856, 0, 'N/A', 0, 'N/A', 0, 0, 0), (200001943954, 0, 'B', 0, 'A', 0, 0, 0), (200001908998, 0, 'N/A', 0, 'N/A', 0, 0, 0), (200001634757, 0, 'A', 0, 'A', 0, 0, 0), (200003132410, 0, 'B', 0, 'A', 0, 0, 0), (200001379770, 0, 'N/A', 0, 'N/A', 0, 0, 0), (200001334237, 0, 'N/A', 0, 'N/A', 0, 0, 0), (200006762554, 0, 'A', 0, 'A', 0, 0, 0), (200005146210, 0, 'A', 0, 'A', 0, 0, 0)]


#### MySQL Create Table

In [12]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dimension_school (
        NU_INSCRICAO BIGINT PRIMARY KEY,
        CO_MUNICIPIO_ESC INT,
        NO_MUNICIPIO_ESC VARCHAR(255),
        CO_UF_ESC INT,
        SG_UF_ESC CHAR(2),
        TP_DEPENDENCIA_ADM_ESC INT,
        TP_LOCALIZACAO_ESC INT,
        TP_SIT_FUNC_ESC INT
    )
""")

In [13]:
conn.commit()

#### MySQL Insert Data

In [14]:
chunk_size = 100000  # Inserir em lotes de 100 mil
for i in range(0, len(data_dimension_school), chunk_size):
    batch = data_dimension_school[i:i+chunk_size]
    cursor.executemany("""
        INSERT IGNORE INTO dimension_school (
            NU_INSCRICAO, CO_MUNICIPIO_ESC, NO_MUNICIPIO_ESC, CO_UF_ESC, SG_UF_ESC, 
        TP_DEPENDENCIA_ADM_ESC, TP_LOCALIZACAO_ESC, TP_SIT_FUNC_ESC
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, batch)
    conn.commit()


In [15]:
cursor.execute("SELECT COUNT(*) FROM dimension_school;")
rows = cursor.fetchall()
print(f"Total de registros na tabela: {rows[0][0]}")

Total de registros na tabela: 5783109


### dimension_exam

#### Data Read

In [16]:
# Leitura do arquivo CSV
data_dimension_exam = []
with open("C:/Users/karen/OneDrive/Desktop/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", mode="r", encoding="latin-1") as file:
    reader = csv.reader(file, delimiter=";")
    next(reader)  # Ignora o cabeçalho

    for row in reader:
        data_dimension_exam.append((
            row[0],  # NU_INSCRICAO
            row[12], # CO_MUNICIPIO_PROVA
            row[13], # NO_MUNICIPIO_PROVA
            row[14], # CO_UF_PROVA
            row[15]  # SG_UF_PROVA
        ))

#### MySQL Create Table

In [17]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dimension_exam (
        NU_INSCRICAO BIGINT PRIMARY KEY,
        CO_MUNICIPIO_PROVA INT,
        NO_MUNICIPIO_PROVA VARCHAR(255),
        CO_UF_PROVA INT,
        SG_UF_PROVA CHAR(2)
    )
""")

In [18]:
conn.commit()

#### MySQL Insert Data

In [24]:
chunk_size = 50000  # Inserir em lotes de 50 mil
for i in range(0, len(data_dimension_exam), chunk_size):
    batch = data_dimension_exam[i:i+chunk_size]
    cursor.executemany("""
        INSERT IGNORE INTO dimension_exam (
            NU_INSCRICAO, CO_MUNICIPIO_PROVA, NO_MUNICIPIO_PROVA, CO_UF_PROVA, SG_UF_PROVA
    )
    VALUES (%s, %s, %s, %s, %s)
    """, batch)
    conn.commit()


OperationalError: 2013 (HY000): Lost connection to MySQL server during query

In [ ]:
cursor.execute("SELECT COUNT(*) FROM dimension_exam;")
rows = cursor.fetchall()
print(f"Total de registros na tabela: {rows[0][0]}")

## Fact Tables

### fact_exam

### MySQL Create Table

#### Data Read

In [ ]:
# Leitura do arquivo CSV
data_fact_exam = []
with open("C:/Users/karen/OneDrive/Desktop/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", mode="r", encoding="latin-1") as file:
    reader = csv.reader(file, delimiter=";")
    next(reader)  # Ignora o cabeçalho

    for row in reader:
        data_fact_exam

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS fact_exam (
        NU_INSCRICAO BIGINT PRIMARY KEY,
        TP_PRESENCA_CN BOOLEAN,
        TP_PRESENCA_CH BOOLEAN,
        TP_PRESENCA_LC BOOLEAN,
        TP_PRESENCA_MT BOOLEAN,
        CO_PROVA_CN INT,
        CO_PROVA_CH INT,
        CO_PROVA_LC INT,
        CO_PROVA_MT INT,
        NU_NOTA_CN FLOAT,
        NU_NOTA_CH FLOAT,
        NU_NOTA_LC FLOAT,
        NU_NOTA_MT FLOAT,
        TX_RESPOSTAS_CN VARCHAR(255),
        TX_RESPOSTAS_CH VARCHAR(255),
        TX_RESPOSTAS_LC VARCHAR(255),
        TX_RESPOSTAS_MT VARCHAR(255),
        TP_LINGUA BOOLEAN,
        TX_GABARITO_CN VARCHAR(255),
        TX_GABARITO_CH VARCHAR(255),
        TX_GABARITO_LC VARCHAR(255),
        TX_GABARITO_MT VARCHAR(255)
    )
""")

In [ ]:
conn.commit()

#### MySQL Insert Data

In [ ]:
chunk_size = 100000  # Inserir em lotes de 100 mil
for i in range(0, len(data_fact_exam), chunk_size):
    batch = data_fact_exam[i:i+chunk_size]
    cursor.executemany("""
        NU_INSCRICAO, TP_PRESENCA_CN, TP_PRESENCA_CH, TP_PRESENCA_LC, TP_PRESENCA_MT,
        CO_PROVA_CN, CO_PROVA_CH, CO_PROVA_LC, CO_PROVA_MT, NU_NOTA_CN, NU_NOTA_CH,
        NU_NOTA_LC, NU_NOTA_MT, TX_RESPOSTAS_CN, TX_RESPOSTAS_CH, TX_RESPOSTAS_LC,
        TX_RESPOSTAS_MT, TP_LINGUA, TX_GABARITO_CN, TX_GABARITO_CH, TX_GABARITO_LC,
        TX_GABARITO_MT
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, batch)
    conn.commit()


In [ ]:
cursor.execute("SELECT COUNT(*) FROM fact_exam;")
rows = cursor.fetchall()
print(f"Total de registros na tabela: {rows[0][0]}")

### fact_composing

#### Data Read

In [ ]:
# Leitura do arquivo CSV
data_fact_composing = []
with open("C:/Users/karen/OneDrive/Desktop/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", mode="r", encoding="latin-1") as file:
    reader = csv.reader(file, delimiter=";")
    next(reader)  # Ignora o cabeçalho

    for row in reader:
        data_fact_composing.append((
            int(row[0]),  # NU_INSCRICAO como BIGINT
            int(row[33]), # TP_STATUS_REDACAO como INT
            int(row[34]), # NU_NOTA_COMP1 como INT
            int(row[35]), # NU_NOTA_COMP2 como INT
            int(row[36]), # NU_NOTA_COMP3 como INT
            int(row[37]), # NU_NOTA_COMP4 como INT
            int(row[38]), # NU_NOTA_COMP5 como INT
            float(row[39]) # NU_NOTA_REDACAO como FLOAT
        ))

#### MySQL Create Table

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS fact_composing (
        NU_INSCRICAO BIGINT PRIMARY KEY,
        TP_STATUS_REDACAO INT,
        NU_NOTA_COMP1 INT,
        NU_NOTA_COMP2 INT,
        NU_NOTA_COMP3 INT,
        NU_NOTA_COMP4 INT,
        NU_NOTA_COMP5 INT,
        NU_NOTA_REDACAO INT
    )
""")

In [ ]:
conn.commit()

#### MySQL Insert Data

In [ ]:
chunk_size = 100000  # Inserir em lotes de 100 mil
for i in range(0, len(data_fact_composing), chunk_size):
    batch = data_fact_composing[i:i+chunk_size]
    cursor.executemany("""
        NU_INSCRICAO, TP_STATUS_REDACAO, NU_NOTA_COMP1, NU_NOTA_COMP2, NU_NOTA_COMP3, 
        NU_NOTA_COMP4, NU_NOTA_COMP5, NU_NOTA_REDACAO
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    """, batch)
    conn.commit()


In [ ]:
cursor.execute("SELECT COUNT(*) FROM fact_composing;")
rows = cursor.fetchall()
print(f"Total de registros na tabela: {rows[0][0]}")

### fact_socioeconomic_questionnaire

#### Data Read

In [ ]:
# Leitura do arquivo CSV
data_fact_socioeconomic_questionnaire = []
with open("C:/Users/karen/OneDrive/Desktop/microdados_enem_2020/DADOS/MICRODADOS_ENEM_2020.csv", mode="r", encoding="latin-1") as file:
    reader = csv.reader(file, delimiter=";")
    next(reader)  # Ignora o cabeçalho

    for row in reader:
        data_fact_socioeconomic_questionnaire.append((
            int(row[0]),  # NU_INSCRICAO como BIGINT
            row[40],      # Q001 como VARCHAR(255)
            row[41],      # Q002 como VARCHAR(255)
            row[42],      # Q003 como VARCHAR(255)
            row[43],      # Q004 como VARCHAR(255)
            row[44],      # Q005 como VARCHAR(255)
            row[45],      # Q006 como VARCHAR(255)
            row[46],      # Q007 como VARCHAR(255)
            row[47],      # Q008 como VARCHAR(255)
            row[48],      # Q009 como VARCHAR(255)
            row[49],      # Q010 como VARCHAR(255)
            row[50],      # Q011 como VARCHAR(255)
            row[51],      # Q012 como VARCHAR(255)
            row[52],      # Q013 como VARCHAR(255)
            row[53],      # Q014 como VARCHAR(255)
            row[54],      # Q015 como VARCHAR(255)
            row[55],      # Q016 como VARCHAR(255)
            row[56],      # Q017 como VARCHAR(255)
            row[57],      # Q018 como VARCHAR(255)
            row[58],      # Q019 como VARCHAR(255)
            row[59],      # Q020 como VARCHAR(255)
            row[60],      # Q021 como VARCHAR(255)
            row[61],      # Q022 como VARCHAR(255)
            row[62],      # Q023 como VARCHAR(255)
            row[63],      # Q024 como VARCHAR(255)
            row[64]       # Q025 como VARCHAR(255)
        ))

#### MySQL Create Table

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS fact_socioeconomic_questionnaire (
        NU_INSCRICAO BIGINT PRIMARY KEY,
        Q001 VARCHAR(255),
        Q002 VARCHAR(255),
        Q003 VARCHAR(255),
        Q004 VARCHAR(255),
        Q005 VARCHAR(255),
        Q006 VARCHAR(255),
        Q007 VARCHAR(255),
        Q008 VARCHAR(255),
        Q009 VARCHAR(255),
        Q010 VARCHAR(255),
        Q011 VARCHAR(255),
        Q012 VARCHAR(255),
        Q013 VARCHAR(255),
        Q014 VARCHAR(255),
        Q015 VARCHAR(255),
        Q016 VARCHAR(255),
        Q017 VARCHAR(255),
        Q018 VARCHAR(255),
        Q019 VARCHAR(255),
        Q020 VARCHAR(255),
        Q021 VARCHAR(255),
        Q022 VARCHAR(255),
        Q023 VARCHAR(255),
        Q024 VARCHAR(255),
        Q025 VARCHAR(255)
    )
""")

In [ ]:
conn.commit()

#### MySQL Insert Data

In [ ]:
chunk_size = 100000  # Inserir em lotes de 100 mil
for i in range(0, len(data_fact_socioeconomic_questionnaire), chunk_size):
    batch = data_fact_socioeconomic_questionnaire[i:i+chunk_size]
    cursor.executemany("""
        NU_INSCRICAO, Q001, Q002, Q003, Q004, Q005, Q006, Q007, Q008, Q009, 
        Q010, Q011, Q012, Q013, Q014, Q015, Q016, Q017, Q018, Q019, 
        Q020, Q021, Q022, Q023, Q024, Q025
    )
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, batch)
    conn.commit()


In [ ]:
cursor.execute("SELECT COUNT(*) FROM fact_socioeconomic_questionnaire;")
rows = cursor.fetchall()
print(f"Total de registros na tabela: {rows[0][0]}")

# Listing the tables

In [ ]:
cursor.execute("SHOW TABLES;")
tables = cursor.fetchall()
print("Tabelas no banco de dados:", tables)

# Tables Structure

In [ ]:
cursor.execute("DESCRIBE dimension_enrollment;")
table_structure = cursor.fetchall()
for column in table_structure:
    print(column)

# Testing the data insert

In [ ]:
cursor.execute("SELECT * FROM dimension_enrollment LIMIT 10;")
rows = print(cursor.fetchall())
print(rows)